# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py gmaps
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from api_keys import g_key


Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#CityData.csv
vacation_data = pd.read_csv('../WeatherPy/CityData.csv')
vacation_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,prince rupert,54.3161,-130.3201,22.32,68,90,5.75,CA,1612922956
1,dwarka,22.2394,68.9678,66.99,75,100,6.20,IN,1612922956
2,rikitea,-23.1203,-134.9692,79.41,69,52,6.31,PF,1612922956
3,paamiut,61.9940,-49.6678,27.23,84,95,10.31,GL,1612922816
4,komsomolskiy,40.4272,71.7189,42.80,81,99,6.91,UZ,1612922957


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = vacation_data[['Lat', 'Lng']]
humidity = vacation_data['Humidity']

fig = gmaps.figure(center=(10,-5),zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                               max_intensity=100,point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_weather = vacation_data[(vacation_data['Max Temp'] < 80) & 
                              (vacation_data['Max Temp'] > 70) &
                              (vacation_data['Wind Speed'] < 10) &
                              (vacation_data['Cloudiness'] == 0) ]
ideal_weather.head()                            

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,el cope,8.6167,-80.6000,77.79,69,0,6.91,PA,1612922963
67,saint-philippe,-21.3585,55.7679,75.40,57,0,6.91,RE,1612922967
69,grand-lahou,5.1367,-5.0242,78.76,90,0,6.26,CI,1612922967
159,alta floresta,-9.8756,-56.0861,75.20,94,0,4.61,BR,1612922981
206,kongoussi,13.3258,-1.5347,73.09,17,0,7.58,BF,1612922989


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = ideal_weather[['Lat','Lng','City','Country']].copy().reset_index().drop(columns='index')
hotel_df['Hotel Name'] = ''
hotel_df

,Lat,Lng,City,Country,Hotel Name
0,8.6167,-80.6000,el cope,PA,
1,-21.3585,55.7679,saint-philippe,RE,
2,5.1367,-5.0242,grand-lahou,CI,
3,-9.8756,-56.0861,alta floresta,BR,
4,13.3258,-1.5347,kongoussi,BF,
5,25.0213,37.2685,umm lajj,SA,
6,12.4850,-16.5469,oussouye,SN,
7,-22.9661,-42.0278,arraial do cabo,BR,
8,-13.7000,-76.2167,pisco,PE,
9,9.9658,98.6348,ranong,TH,


In [6]:
for index, row in hotel_df.iterrows():
    try:
        url ='https://maps.googleapis.com/maps/api/place/nearbysearch/json'
        params = {
         'radius': 5000,
         'keyword': 'hotel',
         'key': g_key }
        latitude = row['Lat']
        longitude = row['Lng']
        params['location'] = f'{latitude},{longitude}'
        res = requests.get(url,params=params).json()
        hotel_df.loc[index, 'Hotel Name'] = res['results'][0]['name']
        if res['results'][0]['name'] == '':
            hotel_df.loc[index, 'Hotel Name'] = 'Na'
        
    except:
        pass
hotel_df

,Lat,Lng,City,Country,Hotel Name
0,8.6167,-80.6000,el cope,PA,CABAÑAS PANAMA
1,-21.3585,55.7679,saint-philippe,RE,Les Embruns Du Baril
2,5.1367,-5.0242,grand-lahou,CI,Hotel Beatitude de Grand Lahou
3,-9.8756,-56.0861,alta floresta,BR,Floresta Amazonica Hotel
4,13.3258,-1.5347,kongoussi,BF,Hôtel La Sagesse
5,25.0213,37.2685,umm lajj,SA,HP Red Sea Hotel
6,12.4850,-16.5469,oussouye,SN,Campement ALULUM
7,-22.9661,-42.0278,arraial do cabo,BR,Orlanova Boutique Hotel
8,-13.7000,-76.2167,pisco,PE,Hostal Mandala
9,9.9658,98.6348,ranong,TH,Sook Hotel


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotels = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotels)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))